# Improved FRCNN

Files uploaded from this model are:
```
JunKai_final_evaluation_frcnn_24000_submission.json
```
 - The numbers represent the training steps done
 - The notebook has went through changes while training


Tutorial adapted from Roboflow - frcnn code
Dataset used: [DeepFashion2 validation dataset](https://github.com/switchablenorms/DeepFashion2)

```
@article{DeepFashion2,
  author = {Yuying Ge and Ruimao Zhang and Lingyun Wu and Xiaogang Wang and Xiaoou Tang and Ping Luo},
  title={A Versatile Benchmark for Detection, Pose Estimation, Segmentation and Re-Identification of Clothing Images},
  journal={CVPR},
  year={2019}
}
```

In [ ]:
!nvidia-smi

Tue Jun 23 15:01:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install tensorflow_gpu==1.15

     |████████████████████████████████| 411.5MB 42kB/s 
     |████████████████████████████████| 3.8MB 45.4MB/s 
     |████████████████████████████████| 512kB 54.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=47735342da709f79b573bb1b708e1aaafd8916974e1575cc6f4bb4839947f1c3
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling te

## Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [ ]:
# If you forked the repo, you can replace the link.
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 200000  # 200000 to improve

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'faster_rcnn_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

## Clone the `tensorflow-object-detection` repository or your fork.

In [ ]:
import os

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
Cloning into 'tensorflow-object-detection-faster-rcnn'...
remote: Enumerating objects: 885, done.
remote: Total 885 (delta 0), reused 0 (delta 0), pack-reused 885
Receiving objects: 100% (885/885), 24.84 MiB | 25.13 MiB/s, done.
Resolving deltas: 100% (413/413), done.
/content/tensorflow-object-detection-faster-rcnn
Already up to date.


## Install required packages

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!pip install tf_slim

!python object_detection/builders/model_builder_test.py

/content
Selecting previously unselected package python-bs4.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpa

##Prepare tfrecord files

In [ ]:
# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
test_record_fname = '/content/drive/My Drive/Colab Notebooks/TIL/CV/input/tfrecord/coco_train.record-00000-of-00001'
# test_record_fname = '/content/drive/My Drive/Colab Notebooks/TIL/CV/DeepFashion2/validation/tf_record/coco_train.record*'
train_record_fname = '/content/drive/My Drive/Colab Notebooks/TIL/CV/DeepFashion2/validation/tf_record/coco_train.record*'
label_map_pbtxt_fname = '/content/drive/My Drive/Colab Notebooks/TIL/CV/input/tfrecord/tfrecord_label_map.pbtxt'

## Download base model

In [ ]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 111M
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 .
drwxr-xr-x 63 root   root 4.0K Jun 23 15:04 ..
-rw-r--r--  1 345018 5000   77 Feb  1  2018 checkpoint
-rw-r--r--  1 345018 5000  55M Feb  1  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 5000  51M Feb  1  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 5000  16K Feb  1  2018 model.ckpt.index
-rw-r--r--  1 345018 5000 5.5M Feb  1  2018 model.ckpt.meta
-rw-r--r--  1 345018 5000 3.2K Feb  1  2018 pipeline.config
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 saved_model


In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/models/research/pretrained_model/model.ckpt'

## Configuring a Training Pipeline

In [ ]:
model_dir = '/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Improved_frcnn_tf_save_folder/'
model_dir = os.path.join('/',*model_dir.split('/'))


print(model_dir)  # model_dir needs to be surrounded by "" when sent to the command line

# Optionally remove content in output model directory to fresh start.
# !rm -rf "{model_dir}"
os.makedirs(model_dir, exist_ok=True)

/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Improved_frcnn_tf_save_folder/


In [ ]:
import re

query = model_dir + "model.ckpt-"
latest_ckpt = !ls "{query}"*

ckpt_num = []
for i in latest_ckpt:
  ckpt_num.append(int(re.findall(r'\d{4,6}', i)[0]))
latest_ckpt = str(max(ckpt_num))

# load checkpoints in. If not resuming from a model, comment this out
fine_tune_checkpoint = os.path.join(model_dir, "model.ckpt-" + latest_ckpt)

fine_tune_checkpoint

'/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Improved_frcnn_tf_save_folder/model.ckpt-28000'

In [ ]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
get_num_classes(label_map_pbtxt_fname)

5

In [ ]:
# Re-writing the pipeline_fname file to know which directory contains what
with open("/content/models/research/object_detection/model_main.py") as f:
    s = f.read()
with open("/content/models/research/object_detection/model_main.py", 'w') as f:
    s = re.sub(re.escape("config = tf.estimator.RunConfig(model_dir=FLAGS.model_dir)"), 
               "config = tf.estimator.RunConfig(model_dir=FLAGS.model_dir, save_checkpoints_steps=2000)", s)
        
    f.write(s)

!cat /content/models/research/object_detection/model_main.py

# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Binary to run train and evaluation on object detection model."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import flags

import tensorflow.compat.v1 as tf

from object_detection import model_hparams
from object_detection import m

In [ ]:
num_classes = get_num_classes(label_map_pbtxt_fname)  # Can actually just sub with no. of classes

# Re-writing the pipeline_fname file to know which directory contains what
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:

    # fixed_shape_resizer - changed as aspect_ratio_resizer causes image shape errors
    # Refer to: https://github.com/tensorflow/tensorflow/issues/34544
    s = re.sub('''keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }''', 
      '''fixed_shape_resizer {
        height: 600
        width: 800
      }''', s)
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # eval_num_examples
    s = re.sub('num_examples: \d*',
               'num_examples: {}'.format(1), s)

    # Set scales - deleted 0.25 (scale*width - 0.25 is for very small objects)
    s = re.sub('scales: \\[.*\\]',
               'scales: [0.5, 1.0, 2.0]', s)
    
    # Set aspect_ratios - deleted 2.0 (width>height)
    s = re.sub('aspect_ratios: \\[.*\\]',
               'aspect_ratios: [0.5, 1.0]', s)
    
    # Number of proposals (previously 300) in second generator
    s = re.sub('max_total_detections: \d*',
               'max_total_detections: {}'.format(150), s)
    
    # Learn rate  
    s = re.sub('step: \d*\n            learning_rate: .00002',
               'step: {}\n            learning_rate: {}'.format(40000, .00004), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s, 1)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # num_readers
    s = re.sub(
        'train_input_reader: {(\n  num_readers: 100)?', r'train_input_reader: {\n  num_readers: 100', s)

    # Evaluation interval
    s = re.sub(
        'eval_config: {(\n  max_evals: \d*)?', r'eval_config: {\n  max_evals: 0', s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)
    
    # Set training steps, num_steps - run indefinitely as one time is only 2000 steps
    s = re.sub('num_steps: [0-9]+',
               '', s)
              #  'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

!cat {pipeline_fname}

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 5
    image_resizer {
      fixed_shape_resizer {
        height: 600
        width: 800
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
     

## Run Tensorboard(Optional)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-06-23 15:07:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.225.3.211, 52.20.175.105, 34.194.84.166, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.225.3.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  17.0MB/s    in 0.8s    

2020-06-23 15:07:19 (17.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
LOG_DIR = model_dir

get_ipython().system_raw(
    'tensorboard --logdir "{}" --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

### Get Tensorboard link

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://cbc68774a1d6.ngrok.io


## Train the model

In [ ]:
# !python -m pip uninstall numpy
# !python -m pip install numpy==1.17

Uninstalling numpy-1.18.5:
  Would remove:
    /usr/bin/f2py
    /usr/bin/f2py3
    /usr/bin/f2py3.6
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.6
    /usr/local/lib/python3.6/dist-packages/numpy-1.18.5.dist-info/*
    /usr/local/lib/python3.6/dist-packages/numpy.libs/libgfortran-ed201abd.so.3.0.0
    /usr/local/lib/python3.6/dist-packages/numpy.libs/libopenblasp-r0-34a18dc3.3.7.so
    /usr/local/lib/python3.6/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.18.5
     |████████████████████████████████| 20.4MB 7.9MB/s 
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requireme

In [ ]:
# Suppress deprecation warnings: https://stackoverflow.com/questions/40426502/is-there-a-way-to-suppress-the-messages-tensorflow-prints/40426709
# import tensorflow as tf
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
import traceback

try:
  !python /content/models/research/object_detection/model_main.py \
      --pipeline_config_path={pipeline_fname} \
      --model_dir="{model_dir}" \
      --alsologtostderr \
      --num_train_steps=100000 \
      --num_eval_steps={num_eval_steps}
except Exception as e:
  traceback.print_exc()

W0623 17:57:14.317968 140359849838464 model_lib.py:717] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 100000
I0623 17:57:14.318217 140359849838464 config_util.py:523] Maybe overwriting train_steps: 100000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0623 17:57:14.318336 140359849838464 config_util.py:523] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0623 17:57:14.318422 140359849838464 config_util.py:523] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0623 17:57:14.318497 140359849838464 config_util.py:523] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I0623 17:57:14.318565 140359849838464 config_util.py:523] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
I0623 17:57:14.318634 140359849838464 config_util.

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://57f4d05da8fb.ngrok.io


In [ ]:
query = model_dir + "model.ckpt-"
latest_ckpt = !ls "{query}"*

ckpt_num = []
for i in latest_ckpt:
  ckpt_num.append(int(re.findall(r'\d{4,6}', i)[0]))
latest_ckpt = str(max(ckpt_num))

# load checkpoints in. If not resuming from a model, comment this out
fine_tune_checkpoint = os.path.join(model_dir, "model.ckpt-" + latest_ckpt)

print(fine_tune_checkpoint)

# Re-writing the pipeline_fname file to know which directory contains what
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    f.write(s)

/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Improved_frcnn_tf_save_folder/model.ckpt-30000


In [ ]:
import traceback

try:
  !python /content/models/research/object_detection/model_main.py \
      --pipeline_config_path={pipeline_fname} \
      --model_dir="{model_dir}" \
      --alsologtostderr \
      --num_train_steps=100000 \
      --num_eval_steps={num_eval_steps}
except Exception as e:
  traceback.print_exc()

W0623 15:39:42.725385 140513253910400 model_lib.py:717] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 100000
I0623 15:39:42.725605 140513253910400 config_util.py:523] Maybe overwriting train_steps: 100000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0623 15:39:42.725703 140513253910400 config_util.py:523] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0623 15:39:42.725780 140513253910400 config_util.py:523] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0623 15:39:42.725857 140513253910400 config_util.py:523] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I0623 15:39:42.725927 140513253910400 config_util.py:523] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
I0623 15:39:42.725994 140513253910400 config_util.

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://57f4d05da8fb.ngrok.io


In [ ]:
query = model_dir + "model.ckpt-"
latest_ckpt = !ls "{query}"*

ckpt_num = []
for i in latest_ckpt:
  ckpt_num.append(int(re.findall(r'\d{4,6}', i)[0]))
latest_ckpt = str(max(ckpt_num))

# load checkpoints in. If not resuming from a model, comment this out
fine_tune_checkpoint = os.path.join(model_dir, "model.ckpt-" + latest_ckpt)
print(fine_tune_checkpoint)

# Re-writing the pipeline_fname file to know which directory contains what
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    f.write(s)

/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Improved_frcnn_tf_save_folder/model.ckpt-32000


In [ ]:
import traceback

try:
  !python /content/models/research/object_detection/model_main.py \
      --pipeline_config_path={pipeline_fname} \
      --model_dir="{model_dir}" \
      --alsologtostderr \
      --num_train_steps=100000 \
      --num_eval_steps={num_eval_steps}
except Exception as e:
  traceback.print_exc()

W0623 16:11:33.513241 139755449464704 model_lib.py:717] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 100000
I0623 16:11:33.513524 139755449464704 config_util.py:523] Maybe overwriting train_steps: 100000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0623 16:11:33.513649 139755449464704 config_util.py:523] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0623 16:11:33.513736 139755449464704 config_util.py:523] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0623 16:11:33.513828 139755449464704 config_util.py:523] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I0623 16:11:33.513919 139755449464704 config_util.py:523] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
I0623 16:11:33.514002 139755449464704 config_util.

In [ ]:
# /content/models/research/training
!ls "{model_dir}"

checkpoint
events.out.tfevents.1592502322.d0689bcbed25
events.out.tfevents.1592504735.2fc38715b6c7
events.out.tfevents.1592537750.a96c07c4ba08
events.out.tfevents.1592548856.6ea3e0d90724
events.out.tfevents.1592554353.5a883e8ab544
events.out.tfevents.1592557332.2bd3a015e27d
events.out.tfevents.1592561699.60e7df0381c7
events.out.tfevents.1592565360.60e7df0381c7
events.out.tfevents.1592568171.60e7df0381c7
events.out.tfevents.1592570694.60e7df0381c7
events.out.tfevents.1592574084.b72f72a14ca8
events.out.tfevents.1592576938.b72f72a14ca8
events.out.tfevents.1592579247.b72f72a14ca8
events.out.tfevents.1592582798.b72f72a14ca8
events.out.tfevents.1592585655.b72f72a14ca8
graph.pbtxt
model.ckpt-20000.data-00000-of-00001
model.ckpt-20000.index
model.ckpt-20000.meta
model.ckpt-22000.data-00000-of-00001
model.ckpt-22000.index
model.ckpt-22000.meta
model.ckpt-24000.data-00000-of-00001
model.ckpt-24000.index
model.ckpt-24000.meta
model.ckpt-26000.data-00000-of-00001
model.ckpt-26000.index
model.ckpt-

## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory="{output_directory}" \
    --trained_checkpoint_prefix="{last_model_path}"

/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Improved_frcnn_tf_save_folder/model.ckpt-36000
Instructions for updating:
Please use `layer.__call__` method instead.
W0623 18:45:23.270033 139959116846976 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0623 18:45:24.726739 139959116846976 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I0623 18:45:24.740018 139959116846976 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0623 18:45:24.740404 139959116846976 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
Instructions for updating:
Use tf.where 

In [ ]:
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


## Download the model `.pb` file

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
!ls -alh {pb_fname}

-rw-r--r-- 1 root root 50M Jun 23 18:45 /content/models/research/fine_tuned_model/frozen_inference_graph.pb


In [ ]:
query = model_dir + "model.ckpt-"
latest_ckpt = !ls "{query}"*|sort -rn|head -1
latest_ckpt = re.search(r'\d{4,6}', latest_ckpt[0])[0]

save_model_dir = "/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/improved_frcnn_inception_v2_" + latest_ckpt + ".pb"
save_model_dir

'/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/improved_frcnn_inception_v2_36000.pb'

In [ ]:
!cp /content/models/research/fine_tuned_model/frozen_inference_graph.pb "{save_model_dir}"